# Notebook for Fatemeh to run Differentiall Accessibility Analysis

Use conda env with snapatac, polars, numpy, scanpy, pandas

In [ ]:
import snapatac2 as snap
import pandas as pd
import numpy as np 
import polars as pl 
import os
import scanpy as sc

In [ ]:
def write_dataset(data, path, sep='\t', header=True, index=True, compression=None):
    """
    See pandas.to_csv documentation - this is just a wrapper
    :param data: data to save
    :param path: str or Path object, path to save
    :param sep:
    :param header:
    :param index:
    :param compression:
    :return:
    """
    from pathlib import Path

    if isinstance(path, str):
        path = Path(path)

    output_dir = path.parents[0]
    output_dir.mkdir(parents=True, exist_ok=True)

    return data.to_csv(
        path, sep=sep, header=header, index=index, compression=compression
    )

## Step 1. Preprocessing of each ATAC object

Unfortunately ATAC data do not look like RNA data, they do not have nice set of genes, same for all samples in the world. We cannot integrate several ATAC objects because they carry different features. That's why we need first to make ATAC features same across different samples. To do this I have this step

In [ ]:
def process_sample(sample_id, gene_matrix_path, atac_fragments_path, output_snap_path, temp_dir="/scratch/alexandra.livanova/INT/tmpdir"):
    """
    Process a single sample for integration.
    
    Parameters:
        sample_id (str): Sample identifier.
        gene_matrix_path (str): Path to the gene matrix `.h5ad` file (your gene matrix with cell annotation)
        atac_fragments_path (str): Path to the atac_fragments.tsv.gz file (also cellranger output)
        output_snap_path (str): Path to save the processed snap `.h5ad` file
        cell_type (str): Cell type to filter in the whitelist
        temp_dir (str): Temporary directory for snap.pp.import_data.
    """

    
    # Read gene matrix with annotated cells
    gene_matrix = sc.read(gene_matrix_path)
    
    # Decompress atac_fragments.tsv.gz

    atac_fragments_decompressed = atac_fragments_path.rstrip('.gz')
    if not os.path.exists(atac_fragments_decompressed):
        print(f"{atac_fragments_decompressed} does not exist. Decompressing...")
        os.system(f"gunzip {atac_fragments_path}")
    else:
        print(f"{atac_fragments_decompressed} already exists. Skipping decompression.")

    
    # Import data using SnapATAC
    print("start creating snap object")
    data = snap.pp.import_data(
        atac_fragments_decompressed,
        chrom_sizes=snap.genome.hg38,
        file=output_snap_path,
        sorted_by_barcode=False,
        whitelist=gene_matrix.obs[gene_matrix.obs['celltype_from_atac']=='Tumor cells'].index.tolist(), # here I select tumor cells only, ATAC object will have smaller size
        tempdir=temp_dir
    )
    
    # Add tile matrix
    snap.pp.add_tile_matrix(data)
    data.close()
    print("snap object is done")
    
    # Reload and add metadata
    data = sc.read(output_snap_path)
    data.obs['patient'] = sample_id # here I add patient id
    data.obs['celltype_from_atac'] = gene_matrix.obs['celltype_from_atac'].reindex(data.obs_names) #here I create a column with cell annotation, will be Tumor cells only in my case
    data.write(output_snap_path) #save your object
    
    # Recompress atac_fragments.tsv
    os.system(f"gzip {atac_fragments_decompressed}")
    print(f"Processing of {sample_id} completed.")

In [ ]:
# Define sample-specific parameters, add your GBM samples (should be 6 of them)
samples = [
                {
            "sample_id": "INT_C102",
        "gene_matrix_path": "/group/sottoriva/alexandra.livanova/INT/multiome/INT_C102_T3/data/INT_C102_T3_gene_matrix.h5ad",
        "atac_fragments_path": "/group/sottoriva/00-PROCESSED_DATA/2023-INT/INT_C102/INT_C102/INT_C102_T/INT_C102_T3/multiome/LAZ_01033/atac_fragments.tsv.gz",
        "output_snap_path": "/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_normal_integration/samples_to_merge/INT_C102_snap.h5ad"
    },  
]

In [ ]:
# Process each sample
for sample in samples:
    process_sample(**sample)

## Step 2. Aggregate your ATAC objects

Now we can aggregate our pre-processed ATAC objects as soon as they contain raw data, have same features in tile matrices, and same obs columns. This step I do in sbatch code as soon as it is impossible to aggregate such huge tables in interactive session

#### In cluster I have a file like this



#!/bin/bash
#SBATCH --job-name=atac_concat       # Job name
#SBATCH --output=atac_concat.log         # Output file
#SBATCH --error=atac_concat.err           # Error file
#SBATCH --time=24:00:00                   
#SBATCH --ntasks=1                     # Number of tasks
#SBATCH --cpus-per-task=32              # Number of CPU cores per task
#SBATCH --mem=256G                       # Memory per node

source ~/.bashrc
conda activate atac_env_310

python atac_concat.py

conda deactivate

#### this file runs python file like this



import anndata as ad
import snapatac2 as snap
import scanpy as sc
import os
from scipy import sparse


data02b2=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_002_B2_snap.h5ad")
data02=sc.read('/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_002_S1_snap.h5ad')
data_06=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_006_S1_snap.h5ad")
data_08=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_008_S1_snap.h5ad")
data_11=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_011_S1_snap.h5ad")
data_14=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_014_B2_snap.h5ad") 
data_21=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_021_B2_snap.h5ad") 
data_25_b2=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_025_B2_snap.h5ad")
data_25=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_025_S1_snap.h5ad")
data_31=sc.read("/group/sottoriva/alexandra.livanova/INT/multiome/CRC_integration/ATAC_UNICORN_integration/samples_to_integrate/U01_031_B2_snap.h5ad")

atac_data=ad.concat([data02b2, data02])
atac_data=ad.concat([atac_data, data_06])
atac_data=ad.concat([atac_data, data_08])
atac_data=ad.concat([atac_data, data_11])
atac_data=ad.concat([atac_data, data_14])
atac_data=ad.concat([atac_data, data_21])
atac_data=ad.concat([atac_data, data_25_b2])
atac_data=ad.concat([atac_data, data_25])
atac_data=ad.concat([atac_data, data_31])

atac_data.write("/group/sottoriva/alexandra.livanova/INT/multiome/unicorn_atac_all_snap2.h5ad")

data = snap.read('/group/sottoriva/alexandra.livanova/INT/multiome/unicorn_atac_all_snap2.h5ad')
        
snap.pp.select_features(data, n_features=25000)
        
snap.tl.spectral(data, n_comps=30)
        
snap.tl.umap(data)

data.close()

data = snap.read('/group/sottoriva/alexandra.livanova/INT/multiome/unicorn_atac_all_snap2.h5ad')

snap.tl.leiden(data)
data.close()

## Step 3. Peak calling and differential accessibility test + motif enrichment

By this moment you have aggregated anndata with all samples together. You can run peak calling and DA analysis

In [ ]:
#change these paths
input_dir = "" #folder with aggregated adata
output_peak_mat_dir = ""
output_diff_peaks_dir = ""
output_motifs_tumor_dir = ""
output_motifs_normal_dir = ""

# create paths if they do not exist
os.makedirs(output_peak_mat_dir, exist_ok=True)
os.makedirs(output_diff_peaks_dir, exist_ok=True)
os.makedirs(output_motifs_tumor_dir, exist_ok=True)
os.makedirs(output_motifs_normal_dir, exist_ok=True)

# There is an adata with uns['reference_sequences'] inside, i have to use it because mine was lost during the aggregation
data_ref=snap.read("/group/sottoriva/alexandra.livanova/UNICORN/multiome/U01_002S1_T3/data/U01_002_snap.h5ad")


# Process all .h5ad files in the input directory

files=[

    'INT_C128_snap_with_normal_all.h5ad' #your aggregated adata in input folder
]
for file in files:
    input_path = os.path.join(input_dir, file)
    print(f"Processing: {file}")

    # Read the dataset and transfer the ref sequences (very stupid thing but must be done)
    data = snap.read(input_path)
    data.uns['reference_sequences']=data_ref.uns['reference_sequences']
    data_ref.close()

    # Call peaks by groups
    print("Peak calling...")
    snap.tl.macs3(data, groupby='celltype_from_atac')
    print("Peak merging...")
    peaks = snap.tl.merge_peaks(data.uns['macs3'], snap.genome.hg38)

    # Create peak matrix
    print("Peak matrix is being created...")
    peak_mat = snap.pp.make_peak_matrix(data, use_rep=peaks['Peaks'])
    peak_mat_path = os.path.join(output_peak_mat_dir, file)
    peak_mat.write(peak_mat_path)

    # Differential peak analysis
    tumor = data.obs['celltype_from_atac'] == "Tumor cells"
    epit = data.obs['celltype_from_atac'] == "Epithelial cells"
    peaks_selected = np.logical_or(
            peaks["Tumor cells"].to_numpy(),
            peaks["Epithelial cells"].to_numpy(),
        )
    data.close()

    print("Diff peaks test is running...")
    diff_peaks = snap.tl.diff_test(
            peak_mat,
            cell_group1=tumor,
            cell_group2=epit,
            features=peaks_selected,
        )

    diff_peaks_path = os.path.join(output_diff_peaks_dir, file)
    write_dataset(pd.DataFrame(diff_peaks, columns=diff_peaks.columns), diff_peaks_path)

    diff_peaks = diff_peaks.filter(pl.col('adjusted p-value') < 0.01) #select significant peaks only

    peaks = {
            "Tumor cells": diff_peaks.filter(pl.col("log2(fold_change)") > 0)['feature name'].to_numpy(),
            "Epithelial cells": diff_peaks.filter(pl.col("log2(fold_change)") < 0)['feature name'].to_numpy(),
        }

    # Motif enrichment analysis
    print("Motif enrichment is running...")
    motifs = snap.tl.motif_enrichment(
            motifs=snap.datasets.cis_bp(unique=True),
            regions=peaks,
            genome_fasta=snap.genome.hg38,
        )

    # Save motifs for tumor cells
    motifs_tumor_cells = pd.DataFrame({
            'index': motifs["Tumor cells"]['name'].to_list(),
            'LFC': motifs["Tumor cells"]['log2(fold change)'].to_list(),
            'padj': motifs["Tumor cells"]['adjusted p-value'].to_list()
        }).set_index('index')

    motifs_tumor_path = os.path.join(output_motifs_tumor_dir, file)
    write_dataset(motifs_tumor_cells, motifs_tumor_path)

    # Save motifs for normal cells
    motifs_normal_cells = pd.DataFrame({
            'index': motifs["Epithelial cells"]['name'].to_list(),
            'LFC': motifs["Epithelial cells"]['log2(fold change)'].to_list(),
            'padj': motifs["Epithelial cells"]['adjusted p-value'].to_list()
        }).set_index('index')

    motifs_normal_path = os.path.join(output_motifs_normal_dir, file)
    write_dataset(motifs_normal_cells, motifs_normal_path)

    # Close the dataset

    print(f"Finished processing: {file}")